In [3]:
import random
import os
import pandas as pd
import numpy as np
import copy
import operator

In [ ]:
# данный код из таблицы создаёт словарь отношений, который далее используется для сортировки.
# Я торопился, поэтому здесь не очень красивый код.

In [995]:
data = pd.read_excel(r'C:\Users\Professional\Downloads\final.xlsx')

In [835]:
# data.drop('Отметка времени', inplace = True, axis=1)

In [836]:
# далее я привожу данные в удобный для обработки вид.

In [996]:
list_names = []
for i in data.iloc[:, 1]:
    if len(i.split()) == 2:
        list_names.append(i)
    else:
        list_names.append(' '.join(i.split()[:-1]))

In [997]:
data['id'] = list(np.arange(1, len(list_names) + 1))

In [998]:
list_names = [' '.join(x.split()[::-1]) for x in list_names]

In [999]:
data.iloc[:, 0] = data.iloc[:, 0].apply(lambda x: int(x[5:]))

In [1000]:
data.iloc[:, 1] = list_names

In [1001]:
data.iloc[:, 2].value_counts()

Нет        87
Никогда    52
Да         20
Name: Хотели бы вы поменять группу?, dtype: int64

In [1002]:
def change_group(x):
    if x == 'Нет':
        return 0
    elif x == 'Да':
        return 1
    elif x == 'Никогда':
        return 2

In [858]:
# прописать группу в 5 столбце

In [1003]:
data.iloc[:, 2] = data.iloc[:, 2].apply(change_group)

In [1004]:
def string_to_list(x):
    l = x.split(',')
    l = [x.strip() for x in l]
    return l

In [1005]:
for i in range(4, 7):
    data.iloc[:, i] = data.iloc[:, i][data.iloc[:, i].notna()].apply(string_to_list)

In [1006]:
special_cols = [3, 7]

In [1007]:
data = data.astype('object')
data.iloc[:, special_cols]
for i in special_cols:
    for j in range(len(data)):
        try:
            if not data.iloc[j, i].isnumeric():
                l = data.iloc[j, i].split(',')
                l = [x.strip() for x in l]
                data.iloc[j, i] = l
        except:
            pass

In [878]:
# data.sample(10)

In [ ]:
# функция ниже получает эту таблицу. Ей также нужен ключ - имя человека.
# Ко мне позже дошло, что лучше было бы заполнить нулями все NaN, было бы лакончинее. Но я изворачивался и писал через try, except. Причём не рекомендуется оставлять голый except
# В функции стоит прописывать коэффициенты, чтобы контролировать чем жертвовать. Все желания людей всё равно не получится реализовать.
# на выходе - словарь отношений для одного человека.
# Выглядит страшно... Извините, пожалуйста. Обещаю меньше писать такой код. Лучше, наверное, разбивать на мелкие функции.

In [1027]:
def get_dict_relation(key, dataAllQuiz = data, cChange = 1.3, cVeryFavGroup = 1.6, cFavGroup = 1, cFavIds = 10, cTogFor = 11, cFavPeople = 7, cCanan = 6, cHaters = -10, cHateGroup = -1):
    dict_one_relation = {}
    id_on_key = int(dataAllQuiz[dataAllQuiz.name == key].id)
#     здесь проверяем хочет ли человек остаться в своей группе.
    if dataAllQuiz.iloc[id_on_key - 1, 2] == 0:
        for i in dataAllQuiz['name'][(dataAllQuiz.group == dataAllQuiz.iloc[id_on_key - 1].group) & (dataAllQuiz.id != dataAllQuiz.iloc[id_on_key - 1].id)]:
            dict_one_relation[i] = cChange
    elif dataAllQuiz.iloc[id_on_key - 1, 2] == 1:
        try:
            if isinstance(data.iloc[id_on_key - 1, 3][0], int):
                for num_group in data.iloc[id_on_key - 1, 3]:
                    for i in dataAllQuiz[dataAllQuiz.group == num_group].name:
                        dict_one_relation[i] = cFavGroup                  
            elif type(data.iloc[id_on_key - 1, 3][0]) is str:
                for people in data.iloc[id_on_key - 1, 3]:
                    dict_one_relation[people] = cFavIds
        except:
            pass
    elif dataAllQuiz.iloc[id_on_key - 1, 2] == 2:
        for i in dataAllQuiz['name'][(dataAllQuiz.group == dataAllQuiz.iloc[id_on_key - 1, 0]) & (dataAllQuiz.id != dataAllQuiz.iloc[id_on_key - 1].id)]:
            dict_one_relation[i] = cVeryFavGroup
# здесь проверяем с кем человек не хотел бы быть. Либо с группой, либо с конкретными людьми.
    for i in dataAllQuiz['listHaters']:
        try:
            if isinstance(data.iloc[id_on_key - 1, 7][0], int):
                for num_group in data.iloc[id_on_key - 1, 7]:
                    for i in dataAllQuiz['name'][(dataAllQuiz.group == num_group) & (dataAllQuiz.id != dataAllQuiz.iloc[id_on_key - 1].id)]:
                        dict_one_relation[i] = cHateGroup
            elif type(data.iloc[id_on_key - 1, 7][0]) is str:
                for people in data.iloc[id_on_key - 1, 7]:
                    dict_one_relation[people] = cHaters
        except:
            pass
# а здесь все остальные значения.
    columns_values = {'listFavPeople':cFavPeople, 'listOfTogFor':cTogFor, 'listCananBe':cCanan}
    for keys, values in columns_values.items():
        try:
            if type(pd.isna(dataAllQuiz.iloc[id_on_key - 1][keys])) is not bool:
                for i in dataAllQuiz.iloc[id_on_key - 1][keys]:
                    dict_one_relation[i] = values
        except:
            pass

    return dict_one_relation

In [1028]:
# создаю и далее выгружаю полный словарь отношений.
full_dict_relations = {}
for i in list_names:
    full_dict_relations[i] = get_dict_relation(i, dataAllQuiz = data)
# длина этого словаря равняется количеству людей.

In [1029]:
with open('full_dict_relationsRazval4.txt', 'w') as f:
    f.write(str(full_dict_relations))
# далее он используется в sort.ipynb 